In [17]:
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
import gensim.corpora as corpora

#Get stopword list from file
def stopwords():
	stopwordList =[]
	with open("stopwords.txt",'r') as File:
		for line in File:
			for word in line.split():
				stopwordList.append(word.lower())
	return stopwordList

stop_words = stopwords()
stop_words.extend(['br'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

def remove_wordsgreaterthan(texts,length):
     return [[word for word in simple_preprocess(str(doc)) 
             if len(word)>length] for doc in texts]

    
data = pd.read_csv('FoodReviews.csv',low_memory = False)# Print head
print(data)

# Remove all columns exclusing the text
data.filter(items=['Text']) #'Id','Summary'

#Preprocess data
# Convert the text to lowercase and get rid of nonalphanumerics and get rid of numbers
data['Text'] = data['Text'].map(lambda x: re.sub('[,.!?@$-<>]', '', x))
data['Text'] = data['Text'].map(lambda x: x.lower())

#filter rows based on length of the review and then choose a sample size
data = data.loc[data['Text'].str.len() > 60].sample(1000)
data.filter('Text')
print(data['Text'])

data = data.Text.values.tolist()
data_words = list(sent_to_words(data))
data_words = remove_stopwords(data_words)
data_words = remove_wordsgreaterthan(data_words,2)

#Using Gensim LDA Model
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]# View
print(corpus[:1][0][:30])

from pprint import pprint
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())


            Id   ProductId          UserId                      ProfileName  \
0            1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1            2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2            3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3            4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4            5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
...        ...         ...             ...                              ...   
568449  568450  B001EO7N10  A28KG5XORO54AY                 Lettie D. Carter   
568450  568451  B003S1WTCU  A3I8AFVPEE8KI5                        R. Sawyer   
568451  568452  B004I613EE  A121AA1GQV751Z                    pksd "pk_007"   
568452  568453  B004I613EE   A3IBEVCTXKNOH          Kathy A. Welch "katwel"   
568453  568454  B001LR2CU2  A3LGQPJCZVL9UC                         srfell17   

        HelpfulnessNumerator  HelpfulnessDenominato

In [18]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [19]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
p

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.020038 -0.002263       1        1  13.608173
5      0.024633 -0.029179       2        1  13.140103
0     -0.002025 -0.007852       3        1  11.490210
8      0.011752  0.019012       4        1  10.523094
9     -0.000298 -0.035330       5        1  10.310197
4      0.012840  0.050000       6        1   9.130632
3     -0.030505  0.006547       7        1   9.027694
6      0.016148 -0.031361       8        1   8.951077
7      0.020905  0.029816       9        1   7.606748
2     -0.073489  0.000610      10        1   6.212072, topic_info=       Term        Freq       Total Category  logprob  loglift
218  coffee  262.000000  262.000000  Default  30.0000  30.0000
326     dog   99.000000   99.000000  Default  29.0000  29.0000
161   great  234.000000  234.000000  Default  28.0000  28.0000
13     good  302.000000  302.000000  Default  27.0000  27.0000
3       bag   82.000000   82.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
13     good    8.824262  302.161067  Topic10  -5.6688  -0.7548
21     make    6.952211  123.923524  Topic10  -5.9073  -0.1019
102    even    6.884237  119.642773  Topic10  -5.9171  -0.0766
363  enough    6.210529   65.754112  Topic10  -6.0201   0.4190
753    dont    7.006035  168.849190  Topic10  -5.8996  -0.4036

[860 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5098      7  0.579234  adjective
4060      9  0.614540   airplane
7074      2  0.577281     alicia
473       1  0.071841  allergies
473       2  0.071841  allergies
...     ...       ...        ...
3966      5  0.109372        yum
3966      8  0.109372        yum
3966     10  0.218744        yum
5644      9  0.406858       zest
4545      1  0.731611       ziwi

[2204 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 1, 9, 10, 5, 4, 7, 8, 3])